In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sps

In [ ]:
xgrid = np.linspace(0., 10., 51)

In [ ]:
nbkg = 40.
nsig = 5*np.sqrt(nbkg)

In [ ]:
def plot_trial(nsig, nbkg):
    x_cent = 0.5*(xgrid[1:] + xgrid[0:-1])
    mask = np.bitwise_and(x_cent > 4.5, x_cent < 5.5)
    bkg_model = nbkg*(10./50.)*np.ones((50))
    sig_model = nsig*(10./50.)*sps.norm(loc=5.0, scale=0.10).pdf(x_cent)
    tot_model = bkg_model + sig_model
    fake_data = np.random.poisson(tot_model)
    _ = plt.bar(x_cent, fake_data, width=0.2)
    plt.xlabel("Variable [a.u.]")
    plt.ylabel("Counts [per 0.2 a.u.]")
    bkg_exp = bkg_model[mask].sum()
    sig_exp = sig_model[mask].sum()
    roi_evt = fake_data[mask].sum()
    pvalue = sps.poisson(mu=bkg_exp).sf(roi_evt)
    signif = sps.norm().isf(pvalue)
    plt.title(r"$n_{\rm ex} = %0.3f$,  $n_{obs} = %i$,  $p=%0.2e$,  $s=%0.1f\sigma$" % (bkg_exp, roi_evt, pvalue, signif))
    print("Expected background: %0.3f" % bkg_exp)
    print("Expected signal:     %0.3f" % sig_exp)
    print("Observed events:     %i" % fake_data[mask].sum())
    print("P-value:             %0.2e" % pvalue)
    

In [ ]:
def plot_5_sigma(nbkg):
    nsig = 5*np.sqrt(nbkg)
    plot_trial(nsig, nbkg)

In [ ]:
plot_5_sigma(10)

In [ ]:
plot_5_sigma(100)

In [ ]:
plot_5_sigma(1000)

In [ ]:
plot_5_sigma(10000)

In [ ]:
plot_5_sigma(100000)